# Get Paper Info
These notebook will get the paper info for each speakers in the conference. The attributes are: Number of papers (by year), first paper year, number of citation (by year), country

In [1]:
%run setup.py

## Load the data

In [20]:
# Append all files in Data folder as a datafrom, and add a column for the file name without the extension and folder name
df_new  = pd.read_csv(os.path.join(filepath,'factProceedings_init.csv'), encoding='utf-8')
df_base  = pd.read_csv(os.path.join(filepath,'factProceedings.csv'), encoding='utf-8')

# Find new names
df_new = df_new[~df_new['Full name'].isin(df_base['Full name'])].reset_index(drop=True)

dblp_name = df_new['Full name'].str.replace(' ', '%20')
dblp_name = dblp_name.drop_duplicates().reset_index(drop=True)

# drop nan values from the list
dblp_name = dblp_name.dropna().reset_index(drop=True)

## Get data from dblp.org

In [21]:
df1 = pd.DataFrame(columns = ['Full name','Year','Year Count'])

for Author in tqdm(dblp_name[:8176]):
    author_list = []
    paper_list = []
    
    ##### Check if author is on dblp #####
    dblp_URL = "https://dblp.org/search?q=" + Author
    page = requests.get(dblp_URL)
    soup = BeautifulSoup(page.content, "html.parser")
    control_check = soup.find(id="completesearch-authors").find_all("ul",class_ = "result-list")
    if len(control_check) > 0:
        first_author = soup.find(id="completesearch-authors").find_all("ul",class_ = "result-list")[0].find_all("a",href=True)[0]['href']
        # print(Author," is on the webpage and the process continues...")
        
        ##### Get info from author #####
        author_page = requests.get(first_author)
        soup_of_speaker = BeautifulSoup(author_page.content, "html.parser")

        Year_list = []

        sections = soup_of_speaker.find(id="publ-section").find_all("div", class_="hide-body")
        for s in range(0,len(sections)): # Nu går vi ind på hver enkelt sektion
            rows_in_sections = sections[s].find_all('ul', class_="publ-list")[0].findAll(True, {"class":['year','entry inproceedings toc','entry article toc','entry incollection toc', 'entry book toc','entry editor toc','entry reference toc']})
            last_row_idx = 0
            
            for row in range(0,len(rows_in_sections)): # Nu går vi ind på hver enkelt række
                #### Append year multipliers
                if rows_in_sections[row].p == None:
                    Year_list.append(rows_in_sections[row].text)
                    paper_list.append(row-last_row_idx-1)
                    last_row_idx = row 
                if row == len(rows_in_sections)-1: # den sidste linje i hver sektion
                    paper_list.append(len(rows_in_sections)-last_row_idx-1)
            
            # remove all -1 values from the list
            paper_list = [x for x in paper_list if x >= 0]
                
        # create a dataframe
        df2 = pd.DataFrame(columns = ['Full name','Year','Year Count'])
        # append full name to the dataframe
        
        df2['Year'] = Year_list
        df2['Year Count'] = paper_list
        df2['Full name'] = Author
        df1 = df1.append(df2, ignore_index=True)
    time.sleep(1)
        

100%|██████████| 8176/8176 [6:47:56<00:00,  2.99s/it]    


In [22]:
df1.to_csv(os.path.join(filepath,"paper_count_per_year_1.csv"), index=False)
df3 = df1.copy()

In [27]:
df3

,Full name,Year,Year Count
0,Isao%20Shirakawa,2016,1
1,Isao%20Shirakawa,2014,2
2,Isao%20Shirakawa,2013,1
3,Isao%20Shirakawa,2011,1
4,Isao%20Shirakawa,2008,1
...,...,...,...
87804,Johanna%20Ylipulli,2017,3
87805,Johanna%20Ylipulli,2016,1
87806,Johanna%20Ylipulli,2015,1
87807,Johanna%20Ylipulli,2014,3


## Combine data and save as factTable

In [28]:
# Load the initial factTable
df  = pd.read_csv(os.path.join(filepath,'factProceedings_init.csv'), encoding='utf-8')
# Load the table with paper count per year
df2 = pd.read_csv(os.path.join(filepath,'paper_count_per_year_proceedings.csv'), encoding='utf-8')
# concat df1 and df2
df1 = pd.concat([df2,df3], axis=0, ignore_index=True)

In [29]:
df1['Full name'] = df1['Full name'].str.replace('%20', ' ')
df1 = df1.drop_duplicates().reset_index(drop=True)
df1['Year'] = df1['Year'].astype(int)
df1['Year Count'] = df1['Year Count'].astype(int)

df1 = df1.sort_values(by=['Year'], ascending=True)

df1['Year Count'] = df1.groupby('Full name')['Year Count'].cumsum()

df1['max_year_count'] = df1.groupby('Full name')['Year Count'].transform('max')
df1['First year paper'] = df1.groupby('Full name')['Year'].transform('min')

In [30]:
# create a function to get the paper count for a given name and year
def get_paper_count(name, year):
    df = df1[df1['Full name'] == name]
    paper_count = df[df['Year'] <= year]['Year Count']
    if len(paper_count) == 0:
        return 0
    return paper_count.iloc[-1]

# merge the dataframes and add a new column with the paper count
merged = pd.merge(df, df1.drop_duplicates(subset=['Full name']), on='Full name', how='left')
merged['Year Count'] = merged.apply(lambda x: get_paper_count(x['Full name'], x['Year_x']), axis=1)

# Clean up and control check
print("Number of rows in original table:",df.shape[0], " and umber of rows in merged table:",merged.shape[0])
print("Pct. of speakers whose information I was able to collect",round((df['Full name'].nunique()/df1['Full name'].nunique())*100,2),"%")
merged['max_year_count'] = merged['max_year_count'].fillna(0)
merged['First year paper'] = merged['First year paper'].fillna(0)
merged = merged.drop(columns=['Year_y'])
merged = merged.rename(columns={'Year_x': 'Year', 'Year Count': 'Paper Count', 'max_year_count': 'Max Paper Count'})
merged.to_csv(os.path.join(filepath, "factInvited.csv"), index=False)
print("The file is now saved")
merged.head(10)

Number of rows in original table: 321681  and umber of rows in merged table: 321681
Pct. of speakers whose information I was able to collect 114.68 %
The file is now saved


,Full name,Year,Conference (short),First name,Middle name,Last name,gender,Paper Count,Max Paper Count,First year paper
0,Isao Shirakawa,1995,ASPDAC,Isao,NaN,Shirakawa,M,26,93.0,1977.0
1,Tingting Hwang,1995,ASPDAC,Tingting,NaN,Hwang,F,16,107.0,1989.0
2,Sanjay Dhar,1995,ASPDAC,Sanjay,NaN,Dhar,M,4,5.0,1983.0
3,Dave J. Gurney,1995,ASPDAC,Dave,J.,Gurney,M,1,1.0,1995.0
4,Michel Thill,1995,ASPDAC,Michel,NaN,Thill,M,3,4.0,1994.0
5,Huy Nam Nguyen,1995,ASPDAC,Huy,Nam,Nguyen,M,6,14.0,1988.0
6,Hiroaki Kunieda,1995,ASPDAC,Hiroaki,NaN,Kunieda,M,1,59.0,1995.0
7,Renato J. O.,1995,ASPDAC,Renato,J.,O.,M,1,128.0,1995.0
8,Vasily G. Moshnyaga,1995,ASPDAC,Vasily,G.,Moshnyaga,M,5,92.0,1992.0
9,Richard K. Wallace,1995,ASPDAC,Richard,K.,Wallace,M,1,1.0,1995.0
